In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

In [ ]:
trainY = train_data.iloc[:,1]
trainX = pd.get_dummies(train_data[["Pclass", "Sex", "Age", "SibSp", "Parch"]])
age_mean = np.mean(trainX['Age'])
trainX['Age'] = trainX['Age'].fillna(age_mean)
trainX

In [ ]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
testX = pd.get_dummies(test_data[["Pclass", "Sex", "Age", "SibSp", "Parch"]])
testX['Age'] = testX['Age'].fillna(age_mean)
testX

paramters of XGBoost: https://xgboost.readthedocs.io/en/stable/parameter.html#parameters-for-tree-booster


In [ ]:
param_grid1 = {'n_estimators':[10,15,20,25,35]}
models = GridSearchCV(XGBClassifier(random_state=1),param_grid1).fit(trainX,trainY)
print(models.best_params_)
model1 = models.best_estimator_

In [ ]:
param_grid2 = {'max_depth':[1,3,5,7,10,15]}
models = GridSearchCV(model1,param_grid2).fit(trainX,trainY)
print(models.best_params_)
model2 = models.best_estimator_

In [ ]:
param_grid3 = {'min_child_weight':[1,2,3,4,5]}
models = GridSearchCV(model2,param_grid3).fit(trainX,trainY)
print(models.best_params_)
model3 = models.best_estimator_

In [ ]:
param_grid4 = {'lambda':[0,0.01,0.1,0.5,1,2,3]}
models = GridSearchCV(model3,param_grid4).fit(trainX,trainY)
print(models.best_params_)
model4 = models.best_estimator_

In [ ]:
param_grid5 = {'learning_rate':[0.03,0.05,0.1,0.3,0.5]}
models = GridSearchCV(model4,param_grid5).fit(trainX,trainY)
print(models.best_params_)
model5 = models.best_estimator_

In [ ]:
print(model5)
print("Training score:")
print(metrics.accuracy_score(trainY, model5.predict(trainX)))

In [ ]:
testY = model5.predict(testX)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': testY})
output = output.astype('int')
output.to_csv('submission.csv', index=False)
print("Finish!")